Whenever using ML:

    1) Design the model (the forward pass), the input size and output size
    2) Construct loss and the optimiser
    3) Training loop
         - Forward pass: Compute prediction
         - Backward pass: Gradients
         - Compute loss for accuracy check
         - Update weights

Without PyTorch

In [ ]:
import numpy as np

# model: y = w*x = 2*x

# Initialisation
x = np.array([1,2,3,4], dtype=np.float32)
y = np.array([2,4,6,8], dtype=np.float32)
w = 0.0

# Forward pass
def forw_pass(x,w):
    y_hat = w*x
    return y_hat

# Loss MSE
def loss(y, y_hat):
    return ((y_hat - y)**2).mean()

# Gradient
def gradient(x,y,y_hat):
    return np.dot(2*x,y_hat-y).mean()

alpha = .01
num_iteration = 15

for epoch in range(num_iteration):
    y_hat = forw_pass(x,w)
    l = loss(y,y_hat)
    dL_dw = gradient(x,y,y_hat)
    w = w - alpha * dL_dw
    print(f"Epoch:{epoch+1}  Loss:{l:.8f}  weight:{w:.5f}")

print(f"\nPrediction after training: f(5) = {forw_pass(5,w):.3f}")
    

Using Autograd of PyTorch

In [56]:
import torch

# model: y = w*x = 2*x

# Initialisation
x = torch.tensor([1,2,3,4], dtype=torch.float32)
y = torch.tensor([2,4,6,8], dtype=torch.float32)
w = torch.tensor(0.0, dtype=torch.float32, requires_grad=True)

# Forward pass
def forw_pass(x):
    return w*x
    
# Loss MSE
def loss(y, y_hat):
    return ((y_hat - y)**2).mean()

# Gradient calc done by autograd
print(f'Prediction before training: f(5) = {forw_pass(5).item():.3f}')
alpha = .01
num_iteration = 100

for epoch in range(num_iteration):
    y_hat = forw_pass(x)
    
    l = loss(y,y_hat)
    l.backward()
    # print(w.grad)

    with torch.no_grad():
        w -= alpha * w.grad
    w.grad.zero_()
    if epoch%10 == 0:
        print(f"Epoch:{epoch+1}  Loss:{l:.8f}  weight:{w:.5f}")    
   

print(f"\nPrediction after training: f(5) = {forw_pass(5):.3f}")
    

Prediction before training: f(5) = 0.000
Epoch:1  Loss:30.00000000  weight:0.30000
Epoch:11  Loss:1.16278565  weight:1.66531
Epoch:21  Loss:0.04506890  weight:1.93411
Epoch:31  Loss:0.00174685  weight:1.98703
Epoch:41  Loss:0.00006770  weight:1.99745
Epoch:51  Loss:0.00000262  weight:1.99950
Epoch:61  Loss:0.00000010  weight:1.99990
Epoch:71  Loss:0.00000000  weight:1.99998
Epoch:81  Loss:0.00000000  weight:2.00000
Epoch:91  Loss:0.00000000  weight:2.00000

Prediction after training: f(5) = 10.000


Using torch.nn module

In [57]:
import torch
import torch.nn as nn
# model: y = w*x = 2*x

# Initialisation
x = torch.tensor([1,2,3,4], dtype=torch.float32)
y = torch.tensor([2,4,6,8], dtype=torch.float32)
w = torch.tensor(0.0, dtype=torch.float32, requires_grad=True)

# Forward pass
def forw_pass(x):
    return w*x
    
alpha = .01
num_iteration = 100

# Loss MSE can be defined by the torch.nn package
loss = nn.MSELoss()
optimiser = torch.optim.SGD([w], lr = alpha)

# Training loop
for epoch in range(num_iteration):
    y_hat = forw_pass(x)
    
    l = loss(y,y_hat)
    l.backward()
    # print(w.grad)

    optimiser.step()
    optimiser.zero_grad()
    
    if epoch%10 == 0:
        print(f"Epoch:{epoch+1}  Loss:{l:.8f}  weight:{w:.5f}")    
   

print(f"\nPrediction after training: f(5) = {forw_pass(5):.3f}")
    

Epoch:1  Loss:30.00000000  weight:0.30000
Epoch:11  Loss:1.16278565  weight:1.66531
Epoch:21  Loss:0.04506890  weight:1.93411
Epoch:31  Loss:0.00174685  weight:1.98703
Epoch:41  Loss:0.00006770  weight:1.99745
Epoch:51  Loss:0.00000262  weight:1.99950
Epoch:61  Loss:0.00000010  weight:1.99990
Epoch:71  Loss:0.00000000  weight:1.99998
Epoch:81  Loss:0.00000000  weight:2.00000
Epoch:91  Loss:0.00000000  weight:2.00000

Prediction after training: f(5) = 10.000


Using torch.nn to define model

In [125]:
import torch
import torch.nn as nn
# model: y = w*x = 2*x

# Initialisation
X = torch.tensor([[1],[2],[3],[4]], dtype=torch.float32)
y = torch.tensor([[2],[4],[6],[8]], dtype=torch.float32)
X_test = torch.tensor([5], dtype = torch.float32)

m, n_features = X.shape
input_size = output_size = n_features
# We dont need w because torch.nn will automatically deal with the parameters

# Model
# model = nn.Linear(input_size, output_size)

# Making a custom model
class LinearRegression(nn.Module):
    def __init__(self,inp_size, out_size):
        super(LinearRegression, self).__init__()
        self.lin = nn.Linear(inp_size, out_size)
    
    def forward(self,x):
        return self.lin(x)

model = LinearRegression(input_size, output_size)
        
print(f"Prediction before training: {model(X_test).item():.3f}\n")

alpha = .11
num_iteration = 100

# Loss MSE can be defined by the torch.nn package
loss = nn.MSELoss()
optimiser = torch.optim.SGD(model.parameters(),lr = alpha )

# Training loop
for epoch in range(num_iteration):
    y_hat = model(X)
    
    l = loss(y,y_hat)
    l.backward()
    # print(w.grad)

    optimiser.step()
    optimiser.zero_grad()
    
    if epoch%10 == 0:
        [w,b] = model.parameters()
        print(f"Epoch:{epoch+1}  Loss:{l:.8f}  weight:{w[0][0].item():.5f}")    
   

print(f"\nPrediction after training: f(5) = {model(X_test).item():.3f}")
    



Prediction before training: -0.646

Epoch:1  Loss:36.50326157  weight:3.62135
Epoch:11  Loss:1.04214954  weight:2.25441
Epoch:21  Loss:0.03020798  weight:2.02907
Epoch:31  Loss:0.00110842  weight:1.99497
Epoch:41  Loss:0.00015794  weight:1.99204
Epoch:51  Loss:0.00006916  weight:1.99357
Epoch:61  Loss:0.00003507  weight:1.99527
Epoch:71  Loss:0.00001794  weight:1.99660
Epoch:81  Loss:0.00000918  weight:1.99756
Epoch:91  Loss:0.00000470  weight:1.99826

Prediction after training: f(5) = 9.997
